# Render Whole Image

In [5]:
import os, numpy as np
import config
from PIL import Image, ImageDraw
import util
from importlib import reload
import dataset as d
reload(d)
reload(util)

de = util.DataExtractor()
gt_df = de.croppedData()


# ------------------------------------------------------------------ constants
SIDE   = config.RES               # target size (square)
S      = config.S
ANCHOR = config.ANCHORS[0]        # (w₀, h₀) in relative units



# ------------------------------------------------------------------ main
def render_gt(gt_df, img_id,
              *, colour=(0, 255, 0, 200),
              out_dir="evaluation"):
    ann = gt_df[gt_df.img_id == img_id]
    if ann.empty:
        raise ValueError(f"no rows with img_id={img_id}")

    img_path = os.path.join(config.img_dir, ann.filename.iloc[0])
    pil = Image.open(img_path).convert("RGB").resize((config.RES, config.RES))

    draw = ImageDraw.Draw(pil, "RGBA")
    for _, row in ann.iterrows():
        draw.rectangle(util._gt_box_from_row(row), outline=colour, width=2)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{img_id}_gt_only.png")
    pil.save(out_path)
    print(f"[render_gt] saved → {out_path}")
    return pil


render_gt(gt_df, img_id=1023)

ModuleNotFoundError: No module named 'config'

# Render  Crop

In [ ]:
import os, numpy as np, config
from PIL import Image, ImageDraw


def render_crop_gt(
        gt_df, img_id,
        *, crop_cells, row_idx, col_idx,
        colour=(255, 0, 0, 200),
        out_dir="evaluation_crops"):
    
    """size       : output edge length (kept 896 for parity)"""

    if S % crop_cells:
        raise ValueError(f"crop_cells={crop_cells} must divide {S}")
    crops_per_dim = S // crop_cells
    if not (0 <= row_idx < crops_per_dim and 0 <= col_idx < crops_per_dim):
        raise ValueError("row_idx / col_idx outside valid range")

    page = gt_df[gt_df.img_id == img_id]
    if page.empty:
        raise ValueError(f"No rows with img_id={img_id}")

    # ---------- crop window in cell coordinates --------------------------------
    cx_min = col_idx * crop_cells
    cx_max = (col_idx + 1) * crop_cells       # exclusive
    cy_min = row_idx * crop_cells
    cy_max = (row_idx + 1) * crop_cells

    crop_boxes = page[(page.cx >= cx_min) & (page.cx < cx_max) &
                      (page.cy >= cy_min) & (page.cy < cy_max)]
    if crop_boxes.empty:
        raise ValueError("No GT boxes inside the requested crop")

    # ---------- prepare full-page canvas (896×896) ------------------------------
    img_path = os.path.join(config.img_dir, crop_boxes.filename.iloc[0])
    full = Image.open(img_path).convert("RGB").resize((config.RES, config.RES))

    # pixel window of the crop on that 896×896 full canvas
    cell_px   = config.RES / config.S                       # width/height of one grid cell
    crop_px   = crop_cells * cell_px           # edge length of the crop region
    left_px   = int(round(col_idx * crop_px))
    top_px    = int(round(row_idx * crop_px))
    right_px  = int(round(left_px + crop_px))
    lower_px  = int(round(top_px  + crop_px))

    # cut & upscale to full 896×896 preview
    crop_img  = full.crop((left_px, top_px, right_px, lower_px)).resize((config.RES, config.RES))
    scale     = config.RES / (right_px - left_px)    # same for x and y

    # ---------- draw ------------------------------------------------------------
    draw = ImageDraw.Draw(crop_img, "RGBA")
    for _, row in crop_boxes.iterrows():
        x0, y0, x1, y1 = util._gt_box_from_row(row)
        # shift into crop coords, then upscale
        box = [(x0 - left_px) * scale,
               (y0 - top_px)  * scale,
               (x1 - left_px) * scale,
               (y1 - top_px)  * scale]
        draw.rectangle(box, outline=colour, width=2)

    # ---------- save / return ---------------------------------------------------
    os.makedirs(out_dir, exist_ok=True)
    name = f"{img_id}_r{row_idx}_c{col_idx}_cells{crop_cells}.png"
    out_path = os.path.join(out_dir, name)
    crop_img.save(out_path)
    print(f"[render_crop_gt] saved → {out_path}")
    return crop_img


render_crop_gt(gt_df,
               img_id=895,
               crop_cells=10,
               row_idx=0,
               col_idx=8)


In [ ]:
reload(d)
reload(config)
reload(util)

cds = d.CroppedDataset(gt_df)

img, target = cds[5]
util.render_crop_from_dataset(img, target)